In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [3]:
dataset=pd.get_dummies(dataset,drop_first=True)


In [4]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [5]:
dataset["Purchased"].value_counts()

0    257
1    143
Name: Purchased, dtype: int64

In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [8]:
independent=dataset[["Gender_Male", "Age", "EstimatedSalary"]]
independent

,Gender_Male,Age,EstimatedSalary
0,1,19,19000
1,1,35,20000
2,0,26,43000
3,0,27,57000
4,1,19,76000
...,...,...,...
395,0,46,41000
396,1,51,23000
397,0,50,20000
398,1,36,33000


In [9]:
dependent=dataset[["Purchased"]]
dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [11]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
param_grid={'kernel':['linear','poly', 'rbf', 'sigmoid'],
         'gamma':['auto','scale'],
             'C':[10,100,1000,2000,3000]} 
grid = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
grid.fit(X_train,Y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [10, 100, 1000, 2000, 3000],
                         'gamma': ['auto', 'scale'],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             scoring='f1_weighted', verbose=3)

In [13]:
grid_predictions=grid.predict(X_test)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(Y_test,grid_predictions)

from sklearn.metrics import classification_report
clf_report=classification_report(Y_test,grid_predictions)

In [14]:
from sklearn.metrics import f1_score
f1_micro=f1_score(Y_test,grid_predictions,average='weighted')
print("The f1_micro value for best parameter{}:".format(grid.best_params_),f1_micro)

The f1_micro value for best parameter{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}: 0.9171245421245421


In [15]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])

0.9654214263661625

In [16]:
print(cm)

[[73  6]
 [ 4 37]]


In [17]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.95      0.92      0.94        79
           1       0.86      0.90      0.88        41

    accuracy                           0.92       120
   macro avg       0.90      0.91      0.91       120
weighted avg       0.92      0.92      0.92       120



In [19]:
re=grid.cv_results_

In [22]:
print(re)

{'mean_fit_time': array([0.02578487, 0.02378421, 0.01978693, 0.01898723, 0.02498407,
       0.02158623, 0.02458477, 0.01738844, 0.06835694, 0.07795186,
       0.02238555, 0.01199155, 0.07975016, 0.06835742, 0.02358494,
       0.01199203, 0.49051447, 0.54471984, 0.0841908 , 0.0125927 ,
       0.5386374 , 0.65037155, 0.07756968, 0.01419039, 0.88533835,
       1.10031142, 0.10516071, 0.01279163, 0.84724936, 1.21712799,
       0.1111299 , 0.01499043, 1.59035234, 1.53482385, 0.14132714,
       0.01459045, 1.57201815, 1.28407612, 0.14176793, 0.01439047]), 'std_fit_time': array([0.00318496, 0.00193883, 0.00444285, 0.00363104, 0.00653847,
       0.00185314, 0.01129949, 0.00667807, 0.00733311, 0.02335171,
       0.00402667, 0.00089474, 0.02243031, 0.01138742, 0.00431452,
       0.00109506, 0.08021836, 0.15795002, 0.02010423, 0.00119961,
       0.11647263, 0.18252682, 0.03912952, 0.00342663, 0.20001346,
       0.19967785, 0.01134325, 0.00097879, 0.18488543, 0.61433406,
       0.01860441, 0.00289

In [20]:
table=pd.DataFrame.from_dict(re)

In [21]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.025785,0.003185,0.003798,3.997841e-04,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.835985,0.782971,0.644599,0.927778,0.890114,0.816289,0.098849,21
1,0.023784,0.001939,0.003798,1.325591e-03,10,auto,poly,"{'C': 10, 'gamma': 'auto', 'kernel': 'poly'}",0.833024,0.799537,0.737557,0.928571,0.888158,0.837369,0.066798,12
2,0.019787,0.004443,0.003198,3.990650e-04,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",0.892857,0.875644,0.841398,0.947015,0.946153,0.900613,0.041029,1
3,0.018987,0.003631,0.002998,6.327113e-04,10,auto,sigmoid,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.769053,0.733523,0.730543,0.753871,0.759910,0.749380,0.014996,32
4,0.024984,0.006538,0.003198,7.478441e-04,10,scale,linear,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}",0.835985,0.782971,0.644599,0.927778,0.890114,0.816289,0.098849,21
5,0.021586,0.001853,0.004198,1.468147e-03,10,scale,poly,"{'C': 10, 'gamma': 'scale', 'kernel': 'poly'}",0.833024,0.799537,0.737557,0.928571,0.888158,0.837369,0.066798,12
6,0.024585,0.011299,0.003997,1.264179e-03,10,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.892857,0.875644,0.841398,0.947015,0.946153,0.900613,0.041029,1
7,0.017388,0.006678,0.003199,3.997144e-04,10,scale,sigmoid,"{'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.769053,0.733523,0.610390,0.753871,0.759910,0.725349,0.058652,40
8,0.068357,0.007333,0.002998,2.611745e-07,100,auto,linear,"{'C': 100, 'gamma': 'auto', 'kernel': 'linear'}",0.835985,0.782971,0.644599,0.927778,0.890114,0.816289,0.098849,21
9,0.077952,0.023352,0.002198,4.001630e-04,100,auto,poly,"{'C': 100, 'gamma': 'auto', 'kernel': 'poly'}",0.833024,0.799537,0.715601,0.928571,0.907658,0.836878,0.076857,14
